In [1]:

import logging
import os
import sys
import tempfile
from glob import glob

import nibabel as nib
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import monai
from monai.data import create_test_image_3d, list_data_collate, decollate_batch
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from monai.transforms import (
    Activations,
    AsChannelFirstd,
    AsDiscrete,
    Compose,
    LoadImaged,
    RandCropByPosNegLabeld,
    RandRotate90d,
    ScaleIntensityd,
    EnsureTyped,
    EnsureType,
)
from monai.visualize import plot_2d_or_3d_image

In [2]:
monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

MONAI version: 0.9.0
Numpy version: 1.23.1
Pytorch version: 1.12.0+cpu
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: af0e0e9f757558d144b655c63afcea3a4e0a06f5
MONAI __file__: C:\Users\sathy\OneDrive\Documents\GitHub\Multi-Modality-Abdominal-Multi-Organ-Segmentation-Challenge-2022\venv\lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 4.0.1
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 9.2.0
Tensorboard version: 2.9.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.64.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.1
pandas version: 1.4.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installi

In [14]:
images = sorted(glob(os.path.join('data/CT_files/imagesTr/', "img*.nii.gz")))
segs = sorted(glob(os.path.join('data/CT_files/labelsTr/', "seg*.nii.gz")))
train_files = [{"img": img, "seg": seg} for img, seg in zip(images[:160], segs[:160])]
val_files = [{"img": img, "seg": seg} for img, seg in zip(images[-40:], segs[-40:])]

In [15]:
print(len(train_files))
print(len(val_files))

160
40


In [19]:
# define transforms for image and segmentation
train_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        AsChannelFirstd(keys=["img", "seg"], channel_dim=-1),
        ScaleIntensityd(keys="img"),
        RandCropByPosNegLabeld(
            keys=["img", "seg"], label_key="seg", spatial_size=[96, 96, 96], pos=1, neg=1, num_samples=4
        ),
        RandRotate90d(keys=["img", "seg"], prob=0.5, spatial_axes=[0, 2]),
        EnsureTyped(keys=["img", "seg"]),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        AsChannelFirstd(keys=["img", "seg"], channel_dim=-1),
        ScaleIntensityd(keys="img"),
        EnsureTyped(keys=["img", "seg"]),
    ]
)

In [20]:
# define dataset, data loader
check_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
# use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
check_loader = DataLoader(check_ds, batch_size=2, num_workers=4, collate_fn=list_data_collate)
check_data = monai.utils.misc.first(check_loader)
print(check_data["img"].shape, check_data["seg"].shape)

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\sathy\OneDrive\Documents\GitHub\Multi-Modality-Abdominal-Multi-Organ-Segmentation-Challenge-2022\venv\lib\site-packages\monai\transforms\transform.py", line 89, in apply_transform
    return _apply_transform(transform, data, unpack_items)
  File "C:\Users\sathy\OneDrive\Documents\GitHub\Multi-Modality-Abdominal-Multi-Organ-Segmentation-Challenge-2022\venv\lib\site-packages\monai\transforms\transform.py", line 53, in _apply_transform
    return transform(parameters)
  File "C:\Users\sathy\OneDrive\Documents\GitHub\Multi-Modality-Abdominal-Multi-Organ-Segmentation-Challenge-2022\venv\lib\site-packages\monai\transforms\croppad\dictionary.py", line 1171, in __call__
    self.randomize(label, fg_indices, bg_indices, image)
  File "C:\Users\sathy\OneDrive\Documents\GitHub\Multi-Modality-Abdominal-Multi-Organ-Segmentation-Challenge-2022\venv\lib\site-packages\monai\tr